## Lab 9

ER190 | Fall 2018

Duncan Callaway<br>
GSI: Seigi Karasaki



Your Name:

Collaborators:

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap

----

## Section 1: A Review of  k-Nearest Neighbors

In homework 5, we used the KNN algortithm for regression -- to predict the PM2.5 levels based on the average of the surrounding k measurements. But this time around, we'll use the same dataset as we did earlier and use KNN to classify the normal or ozone days. This time around, we'll be working with two features, namely the peak wind speed (`WSR_PK`) and the peak temperature (`T_PK`), instead of just one feature.

**Q1.1)** Why can't we use a Bayes classifier in most situations?

**Your answer here**

**Q1.2)** Load the data, stored in a csv named `ozone.csv` in the data folder, into a table. Display the head.

In [ ]:
# your code here

Run the following cell to a see a scatter plot of the data.

In [ ]:
plt.figure(figsize=(10, 7))
plt.ylabel('Peak Temperature')
plt.xlabel('Peak Wind Speed')
for i in range(data.shape[0]):
    if data.Class[i] == 0: # if it's a normal day.
        pltcolor = 'b'
    else:
        pltcolor = 'r'
    plt.scatter(data.WSR_PK[i], data.T_PK[i], c=pltcolor)
plt.legend(['Ozone Day', 'Normal Day']);

Using KNN doesn't seem like a bad idea -- there's only a few cross overs and the possible decision boundary doesn't look too messy. 

Instead of coding the KNN algorithm from scratch like we did in homework 5, we'll make use of scikit-learn's `KNeighborsClassifier`. Check out the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) to see if there are any arguments you to change.

**Q1.3)**

Split the data into training and tests sets using `train_test_split`, with `test_size = 0.33` and `random_state = 1`. Then, instantiate a scikit-learn KNN model and fit the model with `WSR_PK` and `T_PK`. First set n_neighbors to 4 and use the model to predict what kind of "day" it is based on a peak wind speed and temperature of your choice.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = ...

...

Is the class what you expect it to be?

**Your answer here**

**Q1.4)** In what way does KNN function similarly to a Bayes classifier? For reference, the equation used in lecture was 

$$Pr(Y = j | X = x_0) = \frac{1}{K} \sum_{i \in \mathcal{N}_o} I(y_i = j) $$

where $K$ is the number of neighboring training points, and $\mathcal{N}_0$ is the set of $K$ training points closest to observation $x_0$.

The Bayes decision rule is defined as $\hat{y}_i = arg \, \underset{j \in \mathcal{J}}{max} Pr(Y = j | X = x_i)$

***Your Answer Here***

---

Now that we have our classifier fitted, let's test out some values of K. Before we do so, run the cell below, which defines a function that plots the decision boundary for a classifier when given a number of neighbors.

In [ ]:
def plot_boundary(model, X, y, n_neighbors):
    cmap_light = ListedColormap(['#AAAAFF', '#FFAAAA'])
    cmap_bold = ListedColormap(['#0000FF', '#FF0000'])
    h = .02
    
    x_min, x_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
    y_min, y_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(8, 7))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xlabel('Peak Wind Speed')
    plt.ylabel('Peak Temperature')
    plt.title("Ozone/Normal Day Classification (k = %i)"
              % (n_neighbors))
    
    plt.show()

**Q1.4)**

Plot three decision boundaries, using a small balue for K, a large for K, and one somewhere in between. Calculate the scores as well and compare the decision boundaries.

In [ ]:
# ex1
knn1 = KNeighborsClassifier(...)
knn1.fit(...)

...

In [ ]:
# ex2
knn2 = KNeighborsClassifier(...)
knn2.fit(...)

...

In [ ]:
# ex3
knn3 = KNeighborsClassifier(...)
knn3.fit(...)

...

**Q1.5)** Now that we have a a few plots of various decision boundaries, what are some problems using small or large values for K? Reference your plots in your answer.

***Your answer here***

----

## Section 2: Resampling

Resampling methods are an indispensable tool in modern statistics. They involve repeatedly drawing samples from a training set and refitting a model of interest on each sample in order to obtain additional information about the fitted model.

In this lab we will learn about two methods of resampling that you might have seen in Data 8:
* Cross-Validation
* The Bootstrap

**Warm-up:** Suppose that we obtain a bootstrap sample from a set of $n$ observations. When $n$ = 5, what is the probability that the jth observation is in the bootstrap sample? What about when $n$=100? (ISLR 5.2)

**Your answer here**

### Cross-Validation

<img src='cv.png' width="50%" height="50%"></img>

From HW8: Let's discuss what we see in the figure above. We are given a dataset to work with, and do our usual train-test split. We want to set aside the test data until we are absolutely ready to test the model we have created. However, in order to make sure that the model works as planned on data it is not 100% familiar with, we use cross-validation. In the image above, it is split into 4-folds. Essentially, each V that you see above represents a quarter of the training data set. This then acts as the test set, and the model is fitted on the remaining 3/4 of the train. This process is repeated three more times on different subsets, and the average MSE indicates the total validity of the algorithm on the "unseen" data.

----

To gain a little more intuition about cross-validation, we're going to work with the Boston Housing dataset, using two features, `NOX`, the nitrogen oxides concentrations (in ppm) and `LSTAT`, the lower status of the population. Run the following cells to load the data and visualize the two features we'll be working with. 

For more information on the variables, check this [documentation](https://www.kaggle.com/c/boston-housing). target (medv online) should be of particular interest.

In [ ]:
cv_data = pd.read_csv('data/boston_housing.csv')
cv_data.head()

In [ ]:
plt.scatter(cv_data.NOX, cv_data.target);

In [ ]:
plt.scatter(cv_data.LSTAT, cv_data.target);

We'll focus on Leave-One-Out Cross Validation (LOOCV), which involves splitting the set of observations into two parts: a single observation $(x_i, y_i)$ is used for the validation set, and the rest are used for the training set. 

**Q2.1)** What is a drawback with using only one observation for the validation set? Would LOOCV have much utility when splitting large datasets? Explain.

***Your answer here***

----

We'll use scikit-learn's `LeaveOneOut` function to split our dataset. Let's start by splitting our data into training and testing sets. 

**Q2.2)** Split the dataset, using `NOX` and `LSTAT` as our features and `target` as the target variable. As a reminder: don't forget the intercept (we've included it for you here)! Set `test_size` to .25 and `randomstate` to 2.


What does `test_size` do?

**Your answer here**

What does `randomstate` do?

**Your answer here**

In [ ]:
# including the intercept
cv_data['intercept'] =  np.ones(cv_data.shape[0])

X_train, X_test, y_train, y_test = ...

Now, let's fit the model with our training data.

**Q2.3)** Instantiate a `LinearRegression` model and fit the training data. Then, predict the target variable using the test set. Lastly, print the $R^2$ score as well as the MSE of the fitted model. 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

...

print(...)
print(...)

With the scores, we have a baseline to compare a cross-validated score with. Run the following cell to perform LOOCV and check if the $R^2$ score or MSE improves.

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
loo.get_n_splits(X_train)
y_tests = []
y_predictions = []
for train, test in loo.split(X_train):
    Xr_train, Xr_test = np.array(X_train)[train], np.array(X_train)[test]
    yr_train, yr_test = np.array(y_train)[train], np.array(y_train)[test]
    
    lr = LinearRegression()
    lr.fit(Xr_train, yr_train)
    yr_pred = lr.predict(Xr_test)
    
    y_tests.append(yr_test)
    y_predictions.append(yr_pred)

r2 = r2_score(y_tests, y_predictions)
MSE = mean_squared_error(y_tests, y_predictions)
print("R^2 after LOOCV: ", r2)
print("MSE after LOOCV: ", MSE)